In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/My Drive/SemEval/repo/ReCAM/notebooks'

/content/drive/My Drive/SemEval/repo/ReCAM/notebooks


In [3]:
%%capture
!pip install transformers

In [4]:
import sys
sys.path.append('..')
sys.path.append('../src')
import numpy as np
from transformers import AutoTokenizer, AutoModel
from tqdm.auto import tqdm
import torch

In [5]:
# load the dataset

from datasets.concreteness_dataset import ConcretenessDataset

dataset = ConcretenessDataset('../data/imperceptibility/Concreteness Ratings/train/forty.csv',None,)

In [6]:
!nvidia-smi

Sat Dec 12 23:43:35 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 250W |     10MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Word2Vec Embeddings

In [ ]:
# load the GloVe Vectors
from gensim import models
w = models.KeyedVectors.load_word2vec_format('../data/word_vectors/GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
def compute_unk_vec(dataset, word_vec_model):
    i = 0
    vec_avg = 0
    for sample in enumerate(dataset):
        # print(sample)
        word = sample[1][0]
        reg_value = sample[1][1]
        # print(word)
        try:
            words = word.split(' ')
            # print(words)
            vecs = [word_vec_model[word] for word in words]
            vec = sum(vecs)/len(vecs)

            if i==0:
                vec_avg = vec.copy()
            else:
                vec_avg += vec.copy()
            #print(vec_avg)
            i += 1
        except:
            pass
    return i,vec_avg/i
unk_vec = compute_unk_vec(dataset,w)[1]

In [ ]:
def compute_pearson_coefficient(dataset, word_vec_model):
    i = 0
    reg_values = []
    every_dim = [[] for i in range(300)]
    for sample in enumerate(dataset):
        # print(sample)
        word = sample[1][0]
        reg_value = sample[1][1]
        # print(word)
        try:
            words = word.split(' ')
            # print(words)
            vecs = [word_vec_model[word] for word in words]
            vec = sum(vecs)/len(vecs)
            for dim,dim_val in enumerate(vec):
                every_dim[dim].append(dim_val)
            reg_values.append(reg_value)
            i += 1
        except:
            pass
    return every_dim,reg_values
every_dim,reg_values = compute_pearson_coefficient(dataset,w)

In [ ]:
from scipy.stats import pearsonr
corr_values = []
for dim in every_dim:
    corr_values.append(abs(pearsonr(dim, reg_values)[0]))
print(np.max(corr_values))
sorted_dims = list(np.argsort(corr_values))
sorted_dims.reverse()
print(sorted_dims)

0.34990046946299524
[79, 40, 174, 294, 124, 77, 67, 13, 179, 118, 52, 101, 55, 57, 212, 106, 214, 234, 201, 240, 16, 245, 135, 289, 263, 143, 274, 232, 74, 279, 81, 250, 10, 195, 68, 226, 176, 277, 258, 75, 24, 29, 98, 4, 41, 48, 80, 248, 111, 120, 94, 114, 229, 194, 73, 61, 285, 282, 215, 6, 70, 192, 26, 297, 283, 262, 139, 0, 199, 39, 63, 97, 116, 119, 185, 121, 11, 162, 60, 150, 159, 145, 88, 56, 227, 37, 21, 129, 255, 122, 197, 69, 7, 186, 208, 15, 171, 249, 108, 100, 243, 112, 177, 221, 175, 216, 91, 148, 210, 85, 144, 128, 147, 27, 33, 193, 287, 23, 188, 218, 252, 31, 19, 34, 292, 35, 231, 132, 206, 167, 284, 102, 169, 149, 45, 78, 153, 90, 276, 267, 151, 8, 219, 137, 146, 156, 127, 3, 202, 299, 295, 166, 259, 53, 251, 49, 30, 152, 42, 266, 222, 95, 244, 136, 155, 291, 281, 191, 269, 198, 47, 184, 103, 242, 187, 44, 20, 38, 158, 173, 224, 2, 113, 170, 272, 254, 140, 71, 233, 200, 134, 50, 83, 261, 288, 172, 17, 125, 235, 260, 264, 257, 138, 256, 290, 296, 253, 96, 133, 273, 87, 6

In [ ]:
from scipy.stats import spearmanr
corr_values = []
for dim in every_dim:
    corr_values.append(abs(spearmanr(dim, reg_values)[0]))
print(np.max(corr_values))
sorted_dims = list(np.argsort(corr_values))
sorted_dims.reverse()
print(sorted_dims)

0.3443234713308586
[79, 40, 294, 124, 174, 77, 179, 67, 13, 118, 101, 52, 55, 57, 212, 234, 214, 106, 240, 16, 245, 201, 135, 143, 274, 263, 279, 74, 81, 277, 232, 226, 289, 10, 195, 258, 176, 24, 250, 68, 29, 4, 248, 120, 41, 6, 75, 80, 94, 215, 98, 111, 61, 114, 229, 48, 0, 285, 73, 283, 194, 26, 282, 39, 119, 192, 185, 70, 116, 199, 121, 60, 297, 139, 150, 255, 262, 63, 21, 97, 122, 249, 162, 208, 7, 159, 15, 197, 56, 186, 227, 88, 69, 145, 129, 33, 37, 11, 147, 243, 100, 112, 177, 108, 128, 85, 34, 144, 78, 171, 148, 287, 175, 91, 221, 149, 169, 292, 31, 167, 19, 23, 231, 188, 35, 3, 45, 27, 216, 267, 193, 49, 151, 206, 102, 299, 284, 252, 8, 210, 166, 127, 153, 295, 218, 42, 137, 222, 146, 53, 156, 276, 95, 259, 191, 132, 242, 30, 244, 281, 202, 113, 219, 251, 170, 90, 47, 266, 103, 152, 198, 269, 291, 224, 155, 136, 71, 187, 38, 184, 20, 254, 173, 140, 2, 125, 264, 296, 272, 288, 44, 290, 17, 275, 172, 96, 62, 233, 50, 134, 261, 158, 260, 117, 84, 257, 230, 200, 253, 256, 87, 5, 

# BERT Embeddings

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [8]:
def compute_pearson_coefficient(dataset, model_name='bert-base-uncased',dim_of_vec=768):
    i = 0
    reg_values = []
    every_dim = [[] for i in range(dim_of_vec)]

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    model.to(device)
    model.eval()
    for sample in tqdm(enumerate(dataset)):
        # print(sample)
        word = sample[1][0]
        reg_value = sample[1][1]
        # print(word)
        with torch.no_grad():
            inputs = tokenizer(word, return_tensors="pt")
            for key in inputs:
                inputs[key] = torch.tensor(inputs[key],device=device)
            outputs = model(**inputs)
            vec = outputs.last_hidden_state[0][0]

        for dim,dim_val in enumerate(vec):
            every_dim[dim].append(dim_val.cpu().detach().item())
        reg_values.append(reg_value)
        i += 1
    print(i)
    return every_dim,reg_values
every_dim,reg_values = compute_pearson_coefficient(dataset)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



37453


In [9]:
from scipy.stats import pearsonr
corr_values = []
for dim in every_dim:
    corr_values.append(abs(pearsonr(dim, reg_values)[0]))
# del every_dim
# del reg_values
print(np.max(corr_values))
sorted_dims = list(np.argsort(corr_values))
sorted_dims.reverse()
print(sorted_dims)

0.44281709395710567
[411, 8, 721, 9, 687, 419, 249, 193, 518, 741, 652, 119, 177, 175, 87, 446, 558, 90, 323, 458, 21, 595, 287, 286, 331, 474, 516, 585, 56, 103, 127, 395, 447, 498, 657, 62, 290, 157, 573, 460, 238, 608, 711, 83, 4, 542, 276, 448, 377, 355, 146, 528, 751, 301, 74, 190, 674, 394, 571, 26, 186, 98, 726, 71, 131, 277, 387, 188, 761, 722, 191, 434, 228, 580, 106, 525, 256, 240, 431, 86, 543, 320, 373, 701, 378, 250, 649, 490, 688, 638, 540, 344, 413, 339, 665, 565, 44, 202, 340, 709, 48, 392, 444, 536, 453, 266, 520, 450, 315, 294, 356, 505, 502, 168, 409, 753, 602, 436, 118, 31, 139, 173, 629, 731, 707, 348, 720, 464, 747, 204, 544, 160, 556, 717, 243, 691, 442, 93, 663, 666, 37, 259, 455, 677, 101, 553, 234, 401, 541, 599, 397, 197, 471, 136, 328, 507, 198, 6, 262, 165, 360, 182, 533, 534, 352, 574, 187, 305, 23, 18, 615, 627, 648, 501, 695, 457, 195, 96, 357, 639, 767, 503, 479, 50, 358, 488, 55, 140, 27, 5, 745, 20, 232, 284, 684, 59, 415, 600, 735, 421, 269, 369, 590

In [10]:
from scipy.stats import spearmanr
corr_values = []
for dim in every_dim:
    corr_values.append(abs(spearmanr(dim, reg_values)[0]))
print(np.max(corr_values))
sorted_dims = list(np.argsort(corr_values))
sorted_dims.reverse()
print(sorted_dims)

0.4579890008707501
[411, 8, 9, 721, 687, 249, 518, 419, 741, 652, 175, 193, 119, 558, 323, 87, 90, 446, 21, 177, 62, 286, 103, 287, 498, 516, 290, 56, 474, 595, 447, 395, 127, 458, 377, 585, 657, 157, 83, 542, 331, 186, 146, 4, 448, 573, 301, 608, 74, 71, 190, 722, 394, 276, 711, 460, 528, 355, 26, 525, 701, 238, 277, 434, 188, 240, 751, 580, 202, 86, 98, 649, 520, 571, 228, 250, 505, 761, 48, 344, 340, 139, 565, 665, 602, 131, 373, 356, 731, 638, 256, 44, 674, 709, 320, 726, 753, 707, 294, 409, 266, 413, 747, 339, 431, 688, 540, 118, 191, 553, 31, 436, 392, 717, 544, 536, 455, 198, 450, 720, 502, 378, 490, 387, 556, 106, 101, 533, 501, 648, 37, 666, 590, 663, 360, 442, 677, 243, 543, 259, 401, 600, 315, 348, 629, 464, 232, 305, 444, 168, 197, 397, 195, 272, 615, 262, 599, 507, 503, 574, 173, 358, 204, 187, 93, 165, 59, 328, 421, 691, 639, 457, 5, 284, 471, 210, 695, 285, 488, 269, 136, 161, 541, 224, 18, 274, 234, 50, 6, 27, 453, 20, 38, 23, 745, 415, 308, 610, 609, 263, 379, 371, 756